In [20]:
!pip install --upgrade langchain langchain-community chromadb openai pydantic pandas python-dotenv

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 22.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------------- ------------------------ 4.5/11.5 MB 22.3 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.5 MB 23.0 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 20.6 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typ

In [1]:
import os
import re
from typing import List
from dotenv import load_dotenv
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel

In [17]:
openai_api_key = "openai_api_key"  


# Initialize LLM Model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key=openai_api_key,
)

# Initialize OpenAI Embeddings for vector storage
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=openai_api_key,
)

# Define database storage path
db_path = "chroma_db"

# Initialize ChromaDB
db = Chroma(persist_directory=db_path, embedding_function=embeddings)

print("✅ Environment setup complete!")

C:\Users\toto1\AppData\Local\Temp\ipykernel_9740\2662543097.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


✅ Environment setup complete!


In [19]:
# Define a data structure for Listings
class Listing(BaseModel):
    listing_number: int
    neighborhood: str
    price: str
    bedrooms: int
    bathrooms: float
    house_size: str
    description: str
    neighborhood_description: str

# Function to read and parse listings from a file
def parse_listings(file_path: str) -> List[Listing]:
    """Reads and extracts real estate listings from a text file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    raw_listings = content.strip().split("\n\n")  # Split listings by empty lines

    listings = []
    for raw_listing in raw_listings:
        try:
            listing_number = int(re.search(r"Listing (\d+)", raw_listing).group(1))
            neighborhood = re.search(r"Neighborhood: (.+)", raw_listing).group(1)
            price = re.search(r"Price: (.+)", raw_listing).group(1)
            bedrooms = int(re.search(r"Bedrooms: (\d+)", raw_listing).group(1))
            bathrooms = float(re.search(r"Bathrooms: ([\d.]+)", raw_listing).group(1))
            house_size = re.search(r"House Size: (.+)", raw_listing).group(1)
            description = re.search(r"Description: (.+)", raw_listing).group(1).strip()
            neighborhood_description = re.search(r"Neighborhood Description: (.+)", raw_listing).group(1).strip()

            listing = Listing(
                listing_number=listing_number,
                neighborhood=neighborhood,
                price=price,
                bedrooms=bedrooms,
                bathrooms=bathrooms,
                house_size=house_size,
                description=description,
                neighborhood_description=neighborhood_description,
            )
            listings.append(listing)

        except AttributeError:
            print(f"⚠️ Skipping malformed listing:\n{raw_listing}\n")

    return listings

# Load listings from file
listings_file_path = "Listings.txt"
real_estate_listings = parse_listings(listings_file_path)

print(f"✅ Loaded {len(real_estate_listings)} listings from file.")

✅ Loaded 13 listings from file.


In [25]:
def get_documents_for_embedding(listings):
    """Convert listings into vector-storable documents."""
    documents = []
    for listing in listings:
        text = f"""
        Neighborhood: {listing.neighborhood}
        Price: {listing.price}
        Bedrooms: {listing.bedrooms}
        Bathrooms: {listing.bathrooms}
        House Size: {listing.house_size}
        Description: {listing.description}
        Neighborhood Description: {listing.neighborhood_description}
        """
        document = Document(
            page_content=text,
            metadata=listing.dict(),
        )
        documents.append(document)
    return documents

# Check if database is empty and store listings
if db._collection.count() == 0:
    print("📦 Database is empty. Adding listings...")

    # Convert listings into documents
    documents = get_documents_for_embedding(real_estate_listings)

    # Store them in ChromaDB
    db = Chroma.from_documents(
        documents,
        OpenAIEmbeddings(openai_api_key=openai_api_key),
        persist_directory=db_path
    )

    print(f"✅ Database now contains {db._collection.count()} listings.")
else:
    print(f"✅ Database already contains {db._collection.count()} listings.")


📦 Database is empty. Adding listings...


C:\Users\toto1\AppData\Local\Temp\ipykernel_9740\2065652175.py:16: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  metadata=listing.dict(),


✅ Database now contains 13 listings.


In [27]:
# Define questions for user preferences
questions = [
    "How big do you want your house to be?",
    "What are the 3 most important things for you in choosing this property?",
    "Which amenities would you like?",
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",
]

# Example user responses
user_answers = [
    "A spacious four-bedroom house with an open-concept layout and a home office.",
    "Proximity to nature, a vibrant community with social activities, and a well-maintained property.",
    "A swimming pool, a gourmet kitchen, and a home theater.",
    "Access to a reliable train station, easy highway access, and walkable streets.",
    "Suburban, with a quiet atmosphere but within a short drive to urban conveniences like shopping centers and restaurants."
]

print("✅ User preferences collected successfully.")

✅ User preferences collected successfully.


In [29]:
def search_listings(user_preferences) -> List[Listing]:
    """Execute a similarity search in ChromaDB to find matching listings."""
    retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5},
    )
    documents = retriever.invoke(user_preferences)
    return [Listing(**doc.metadata) for doc in documents]

# Search for similar listings
matched_listings = search_listings(" ".join(user_answers))
print(f"✅ Found {len(matched_listings)} matching listings.")

✅ Found 5 matching listings.


In [31]:
def create_personalized_description(listing: Listing, user_preferences) -> str:
    """Generate a personalized description for the listing based on user preferences."""
    prompt = f"""
    User Preferences:
    {user_preferences}

    Original Listing:
    Neighborhood: {listing.neighborhood}
    Price: {listing.price}
    Bedrooms: {listing.bedrooms}
    Bathrooms: {listing.bathrooms}
    House Size: {listing.house_size}
    Description: {listing.description}
    Neighborhood Description: {listing.neighborhood_description}

    Generate a personalized property description highlighting features relevant to the user.
    """

    response = llm.invoke(prompt)
    return response.content.strip()

# Generate personalized descriptions
for listing in matched_listings[:3]:
    print("🏡 Personalized Listing:")
    print(create_personalized_description(listing, user_answers))
    print("\n")

🏡 Personalized Listing:
**Property Description:**

Welcome to your dream home in the tranquil neighborhood of Sunset Valley! This spacious and modern 4-bedroom, 3-bathroom residence offers an open-concept layout, ideal for both family living and entertaining. With 3,500 sqft of luxurious space, you'll find a perfect balance of comfort and style.

Step inside to discover a gourmet kitchen equipped with state-of-the-art appliances, sure to impress any culinary enthusiast. The living area seamlessly flows into a large backyard oasis featuring a sparkling swimming pool and an outdoor kitchen, making it the perfect setting for summer gatherings and social activities with friends and family.

For those who work from home, this property includes a dedicated home office space, allowing for productivity in a serene environment. The master suite boasts a private balcony where you can enjoy stunning mountain views and unwind after a long day.

Sunset Valley is not only an upscale community but al